## Setup

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 37.4MB/s 
     |████████████████████████████████| 1.2MB 34.6MB/s 
     |████████████████████████████████| 204kB 48.7MB/s 
     |████████████████████████████████| 51kB 7.7MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 51kB 8.3MB/s 
Mounted at /content/gdrive


In [2]:
from fastbook import *

## Importing data and defining DataLoaders object

In [3]:
path = untar_data(URLs.MNIST_SAMPLE)

In [28]:
train_3_tensor = torch.stack([tensor(Image.open(o)) for o in (path/'train'/'3').ls()]).float()/255
train_7_tensor = torch.stack([tensor(Image.open(o)) for o in (path/'train'/'7').ls()]).float()/255
valid_3_tensor = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'3').ls()]).float()/255
valid_7_tensor = torch.stack([tensor(Image.open(o)) for o in (path/'valid'/'7').ls()]).float()/255

In [29]:
train_x = torch.cat([train_3_tensor, train_7_tensor]).view(-1, 28*28)
train_y = tensor([1]*len(train_3_tensor) + [0]*len(train_7_tensor)).unsqueeze(1)

valid_x = torch.cat([valid_3_tensor, valid_7_tensor]).view(-1, 28*28)
valid_y = tensor([1]*len(valid_3_tensor) + [0]*len(valid_7_tensor)).unsqueeze(1)

In [30]:
train_dset = list(zip(train_x, train_y))
train_dl = DataLoader(train_dset, batch_size=256)

valid_dset = list(zip(valid_x, valid_y))
valid_dl = DataLoader(valid_dset, batch_size=256)

In [31]:
dls = DataLoaders(train_dl, valid_dl)

## Defining the Neural Network

In [46]:
neural_network = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
)

## Defining the Loss Function

In [34]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets == 1, 1 - predictions, predictions).mean()

## Defining the Optimizer Function

In [35]:
class Optim:
    def __init__(self, params, lr):
        self.params = list(params)
        self.lr = lr
    
    def step(self):
        for p in self.params:
            p.data -= self.lr * p.grad

    def zero_grad(self):
        for p in self.params:
            p.grad = None

## Defining Learner

In [39]:
class Learner:
    def __init__(self, dataloaders, model, opt_class, loss_function, learning_rate=0.1):
        self.train_dl = dataloaders[0]
        self.valid_dl = dataloaders[1]
        self.model = model
        self.opt_function = opt_class(self.model.parameters(), learning_rate)
        self.loss_function = loss_function

    def train_epoch(self):
        for xb, yb in self.train_dl:
            predictions = self.model(xb)
            loss = self.loss_function(predictions, yb)
            loss.backward()
            self.opt_function.step()
            self.opt_function.zero_grad()
    
    def batch_accuracy(self, predictions, yb):
        predictions = predictions.sigmoid()
        correct = (predictions>0.5) == yb
        return correct.float().mean()
    
    def validate_epoch(self):
        accuracy = [self.batch_accuracy(self.model(xb), yb) for xb, yb in self.valid_dl]
        return round(torch.stack(accuracy).mean().item(),4)

    def fit(self, n):
        for i in range(n):
            self.train_epoch()
            print(self.validate_epoch())

## Putting it all together

In [47]:
lr = 0.1
learn = Learner(dls, neural_network, Optim, mnist_loss, lr)

In [48]:
learn.fit(40)

0.5068
0.8149
0.9223
0.9419
0.9619
0.9648
0.9663
0.9682
0.9687
0.9692
0.9707
0.9716
0.9736
0.9741
0.9751
0.9755
0.977
0.9775
0.9775
0.978
0.978
0.979
0.9795
0.9804
0.9804
0.9804
0.9804
0.9814
0.9819
0.9819
0.9819
0.9819
0.9824
0.9824
0.9824
0.9824
0.9824
0.9824
0.9824
0.9824
